**COVID-19 quantiitative analysis using QRMine - Part 2** [See Part 1](https://www.kaggle.com/beapen/qualitative-analysis-of-covid-19)

> QRMine /ˈkärmīn/ is a suite of qualitative research (QR) data mining tools in Python using Natural Language Processing (NLP) and Machine Learning (ML).

Cite QRMine: [Full Text](https://arxiv.org/abs/2003.13519v1)

```
@article{eapenbr2019qrmine,
  title={QRMine: A python package for triangulation in Grounded Theory},
  author={Eapen, Bell Raj and Archer, Norm and Sartpi, Kamran},
  journal={arXiv preprint arXiv:2003.13519 },
  year={2020}
}

```

In [24]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

In [25]:
#df = pd.read_excel('/kaggle/input/covid19/dataset.xlsx')
df = pd.read_excel('/home/bell/Documents/work/data/covid19-dataset.xlsx')
df.head(5)

,Patient ID,Patient age quantile,SARS-Cov-2 exam result,"Patient addmited to regular ward (1=yes, 0=no)","Patient addmited to semi-intensive unit (1=yes, 0=no)","Patient addmited to intensive care unit (1=yes, 0=no)",Hematocrit,Hemoglobin,Platelets,Mean platelet volume,...,Hb saturation (arterial blood gases),pCO2 (arterial blood gas analysis),Base excess (arterial blood gas analysis),pH (arterial blood gas analysis),Total CO2 (arterial blood gas analysis),HCO3 (arterial blood gas analysis),pO2 (arterial blood gas analysis),Arteiral Fio2,Phosphor,ctO2 (arterial blood gas analysis)
0,44477f75e8169d2,13,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,126e9dd13932f68,17,negative,0,0,0,0.236515,-0.02234,-0.517413,0.010677,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,a46b4402a0e5696,8,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,f7d619a94f97c45,5,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,d9e41465789c2b5,15,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
df_cleaned = pd.concat([df['Patient ID'], df['Eosinophils'], df['Leukocytes'], df['Hematocrit'], df['Hemoglobin'], df['Platelets'], df['SARS-Cov-2 exam result']], axis = 1)
df_cleaned.head(5)

,Patient ID,Eosinophils,Leukocytes,Hematocrit,Hemoglobin,Platelets,SARS-Cov-2 exam result
0,44477f75e8169d2,NaN,NaN,NaN,NaN,NaN,negative
1,126e9dd13932f68,1.482158,-0.09461,0.236515,-0.02234,-0.517413,negative
2,a46b4402a0e5696,NaN,NaN,NaN,NaN,NaN,negative
3,f7d619a94f97c45,NaN,NaN,NaN,NaN,NaN,negative
4,d9e41465789c2b5,NaN,NaN,NaN,NaN,NaN,negative


# The last one is the DV and must be binary

In [27]:
    # rename columns
    df_cleaned.columns = ['id', 'eos', 'leu', 'hct', 'hb', 'pt', 'covid']
    df_cleaned.head(5)

,id,eos,leu,hct,hb,pt,covid
0,44477f75e8169d2,NaN,NaN,NaN,NaN,NaN,negative
1,126e9dd13932f68,1.482158,-0.09461,0.236515,-0.02234,-0.517413,negative
2,a46b4402a0e5696,NaN,NaN,NaN,NaN,NaN,negative
3,f7d619a94f97c45,NaN,NaN,NaN,NaN,NaN,negative
4,d9e41465789c2b5,NaN,NaN,NaN,NaN,NaN,negative


In [28]:
# encode DV
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df_cleaned2 = df_cleaned
# Fit the encoder to the pandas column\n",
le.fit(df_cleaned2['covid'])
df_cleaned2['covid'] = le.transform(df_cleaned2['covid'])
df_cleaned2.head(5)

,id,eos,leu,hct,hb,pt,covid
0,44477f75e8169d2,NaN,NaN,NaN,NaN,NaN,0
1,126e9dd13932f68,1.482158,-0.09461,0.236515,-0.02234,-0.517413,0
2,a46b4402a0e5696,NaN,NaN,NaN,NaN,NaN,0
3,f7d619a94f97c45,NaN,NaN,NaN,NaN,NaN,0
4,d9e41465789c2b5,NaN,NaN,NaN,NaN,NaN,0


In [29]:
df_cleaned2.fillna(df_cleaned2.mean(), inplace=True)
#df_cleaned2.dtypes
df_cleaned2['covid'] = df_cleaned2['covid'].astype('bool')
df_cleaned2.to_csv('/tmp/qrmine-quan.csv', index = False)
df_cleaned2.head(5)


,id,eos,leu,hct,hb,pt,covid
0,44477f75e8169d2,7.206147e-09,6.215833e-09,-2.186214e-09,-1.601319e-08,-3.535004e-10,False
1,126e9dd13932f68,1.482158e+00,-9.461035e-02,2.365154e-01,-2.234027e-02,-5.174130e-01,False
2,a46b4402a0e5696,7.206147e-09,6.215833e-09,-2.186214e-09,-1.601319e-08,-3.535004e-10,False
3,f7d619a94f97c45,7.206147e-09,6.215833e-09,-2.186214e-09,-1.601319e-08,-3.535004e-10,False
4,d9e41465789c2b5,7.206147e-09,6.215833e-09,-2.186214e-09,-1.601319e-08,-3.535004e-10,False


In [30]:
#!pip install qrmine==3.4.0

In [31]:
#!pip install scikit-learn==0.22.0


In [32]:
from qrmine import MLQRMine


# Check the accuracy of a NN model

In [38]:
ml = MLQRMine()
ml.csvfile = "/tmp/qrmine-quan.csv"
ml.read_csv()
ml.epochs = 5
ml.prepare_data(True)  # Oversample
ml.get_nnet_predictions()
print("\n%s: %.2f%%" % (ml.model.metrics_names[1], ml.get_nnet_scores()[1] * 100))

Epoch 1/5
 - 2s - loss: 0.6760 - accuracy: 0.5438
Epoch 2/5
 - 1s - loss: 0.6630 - accuracy: 0.5509
Epoch 3/5
 - 1s - loss: 0.6610 - accuracy: 0.5524
Epoch 4/5
 - 1s - loss: 0.6606 - accuracy: 0.5512
Epoch 5/5
 - 1s - loss: 0.6596 - accuracy: 0.5518
10172/10172 [==============================] - 0s 22us/step

accuracy: 55.37%


In [34]:
knn = ml.knn_search(3, 3)
for n in knn:
    print(n)

[ 973 3195 1935]


# Do a PCA

In [35]:
ml.get_pca(3)


Covariance matrix: 
[[ 1.00009832  0.13307855 -0.08312442 -0.10680748  0.33646359]
 [ 0.13307855  1.00009832 -0.17655434 -0.18627464  0.56306504]
 [-0.08312442 -0.17655434  1.00009832  0.96718446 -0.21717888]
 [-0.10680748 -0.18627464  0.96718446  1.00009832 -0.25116269]
 [ 0.33646359  0.56306504 -0.21717888 -0.25116269  1.00009832]]
Eigenvectors 
[[ 0.22931247  0.39342423  0.84851808 -0.26937879  0.00923581]
 [ 0.36420773  0.47394309 -0.51350117 -0.61559075 -0.00975952]
 [-0.55929709  0.43521022 -0.03013473  0.04046004 -0.70372844]
 [-0.56958049  0.41221234 -0.04326698  0.00521498  0.70976039]
 [ 0.42134706  0.51114418 -0.11641942  0.73947203  0.02873882]]

Eigenvalues 
[2.2977289  1.39369153 0.88179711 0.39513878 0.03213527]
Eigenvalues in descending order:
2.2977288999640493
1.3936915349506926
0.8817971057688561
0.39513878328175256
0.0321352697817071
Variance explained:  [45.95006024682213, 27.87109044825624, 17.634208342066067, 7.901998751000002, 0.6426422118555533]
Cumulative:  [ 

# Create 3 clusters and see the profile

In [36]:
ml.prepare_data()
ml.get_kmeans(3)

Cluster:  0
Cluster Length:  5328
Cluster Members
                   id           eos           leu           hct            hb  \
0     44477f75e8169d2  7.206147e-09  6.215833e-09 -2.186214e-09 -1.601319e-08   
1     126e9dd13932f68  1.482158e+00 -9.461035e-02  2.365154e-01 -2.234027e-02   
2     a46b4402a0e5696  7.206147e-09  6.215833e-09 -2.186214e-09 -1.601319e-08   
3     f7d619a94f97c45  7.206147e-09  6.215833e-09 -2.186214e-09 -1.601319e-08   
4     d9e41465789c2b5  7.206147e-09  6.215833e-09 -2.186214e-09 -1.601319e-08   
...               ...           ...           ...           ...           ...   
5638  21af7c93c7fc540  7.206147e-09  6.215833e-09 -2.186214e-09 -1.601319e-08   
5639  ae66feb9e4dc3a0  7.206147e-09  6.215833e-09 -2.186214e-09 -1.601319e-08   
5640  517c2834024f3ea  7.206147e-09  6.215833e-09 -2.186214e-09 -1.601319e-08   
5641  5c57d6037fe266d  7.206147e-09  6.215833e-09 -2.186214e-09 -1.601319e-08   
5642  c20c44766f28291  7.206147e-09  6.215833e-09 -2.186214

array([0, 0, 0, ..., 0, 0, 2], dtype=int32)

## [QRMine](https://arxiv.org/abs/2003.13519v1) is for triangulation (combining qualitative and quantitative data for sensemaking). See the qualitative analysis [here](https://www.kaggle.com/beapen/qrmine-for-covid-19-part-1)

* By [Bell Eapen](https://nuchange.ca)